## Import useful libraries

In [1]:
from bs4 import BeautifulSoup

## Read html an parse cells line by line

In [2]:
with open ('my-iso-code.txt','r') as fp: #'r' Lecture
    html = fp.read()
    soup = BeautifulSoup(html,'html5lib') #Récuper les données plus facilement

In [3]:
#len(soup.find_all('table'))
#'tr' == ligne
#'td' == cellule

In [4]:
trs = soup.find_all('tr') #'trs' comprend l'ensemble des lignes du tableau de l'html 'tr'
print(len(trs))
for tr in trs:
    values = [td.string for td in tr.find_all('td')]
    print(values)

249


In [5]:
#for tr in trs:
#    for td in tr.find_all('td'):
#        print(td)

## Request worldbank!

In [6]:
import pickle
import pandas
import requests

In [7]:
indicator_list={
'IMF WB\nCustoms and other import duties (% of tax revenue) 2016':'GC.TAX.IMPT.ZS',
'WB IMF Non-performing loans to total gross loans ratio (%)':'FB.AST.NPER.ZS',
'UIC Rail freight (mn ton x km travelled)':'IS.RRS.GOOD.MT.K6',
'WB Household final consumption expenditure, PPP bn USD':'NE.CON.PRVT.PP.CD',
'WB UNESCO R&D Expenditure % of GDP':'GB.XPD.RSDV.GD.ZS',
'WB UNESCO Researchers p. mn inhabitants':'SP.POP.SCIE.RD.P6',
'IFDI Net Inflows (% of GDP)':'BX.KLT.DINV.WD.GD.ZS',
'WHO Current Health Expenditure p.c., PPP USD':'SH.XPD.CHEX.PP.CD',
'WB Research Expenditure % of GDP':'GB.XPD.RSDV.GD.ZS',
'WIPO Trademark applications':'IP.TMK.RSCT',
'WIPO Patent Applications':'IP.PAT.RESD'
}

In [8]:
def get_indicator(country_code, indicator_code):
    url = 'http://api.worldbank.org/v2/country/'
    url += country_code
    url += '/indicator/'
    url += indicator_code
    url += '?format=json'
    r = requests.get(url)
    try:
        data = r.json()
        if data[1] is None:
            return None
        else:
            for i in data[1]:
                if i['value'] is None:
                    continue
                else:
                    return i['value']
    except KeyError:
        return None
    except IndexError:
        return None

In [9]:
filename = 'C:\\Users\\Cyril\\Documents\\GitHub\\US02_Python\\Data-sheet-STUDENTS.xlsx'
df_v2 = pandas.read_excel(filename, header = 2, usecols = 'B:AR')

In [12]:
list = ['Country (according to CN Gov):']
for key in indicator_list:
    indicator_code = indicator_list[key]
    list.append(key) #Créer la liste des entêtes
    a = 0 #La variable permet d'incrémenter les lignes du tableau
    print(key)
    for tr in trs: #Pour chaque la ligne dans le tableau
        values = [td.string for td in tr.find_all('td')]
        country_name = str(values[0])
        print(country_name)
        df_v2.at[a, 'Country (according to CN Gov):'] = country_name
        country_code = str(values[3])
        value = get_indicator(country_code, indicator_code)
        df_v2.at[a, key] = value
        if value is not None:
            df_v2.at[a, key] = value
        a = a + 1

IMF WB
Customs and other import duties (% of tax revenue) 2016
Afghanistan
Ã…land Islands
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas (the)
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia (Plurinational State of)
Bonaire, Sint Eustatius and Saba
Bosnia and Herzegovina
Botswana
Bouvet Island
Brazil
British Indian Ocean Territory (the)
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Cayman Islands (the)
Central African Republic (the)
Chad
Chile
China
Christmas Island
Cocos (Keeling) Islands (the)
Colombia
Comoros (the)
Congo (the Democratic Republic of the)
Congo (the)
Cook Islands (the)
Costa Rica
CÃ´te d'Ivoire
Croatia
Cuba
CuraÃ§ao
Cyprus
Czechia
Denmark
Djibouti
Dominica
Dominican Republic (the)
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Falkland Islands (the) [Malvinas]
Faroe Isla

In [13]:
df_v2 = df_v2[list]

In [14]:
#Enregistre un fichier Data_v2 avec les données du tableau df
with open ('C:\\Users\\Cyril\\Documents\\GitHub\\US02_Python\\Data_v2.pickle','wb+') as fp:
    pickle.dump(df_v2,fp)

In [15]:
#Charge le fichier Data_v2
with open ('C:\\Users\\Cyril\\Documents\\GitHub\\US02_Python\\Data_v2.pickle','rb') as fp:
    df_v2 = pickle.load(fp)

In [16]:
#Ecrit un fichier excel
df_v2.to_excel('final-result_v2.xls')